# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
sys.path.append('../../src')

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), True)  # Load sampling strategies


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-3362000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-2082000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/righty_suit-

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = True, True

# You sit North and hold:
hand = 'J.32.KJ983.QJT74'

auction = ["PAD_START","2C"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, 0, 0, True)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

Setting seed (Sampling bidding info) from J.32.KJ983.QJT74: 2930407879
bid 3D value 0.7947 is recommended by NN
CandidateBid(bid=3D  , insta_score=0.7947, expected_score=---, adjust=---)
CandidateBid(bid=3C  , insta_score=0.1046, expected_score=---, adjust=---)
CandidateBid(bid=2N  , insta_score=0.0962, expected_score=---, adjust=---)
Sampling for aution: ['PAD_START', '2C'] trying to find 5000
sample_cards_auction, nsteps= 1
NS:  1.0 EW:  99.0 Auction:  ['PAD_START', '2C']
nesw_i 0
bids in model 4
Player:  S Hand:  J.xx.KJ98x.QJTxx
HCP:  [[ 9.84 11.2  13.07]]
Shape:  [[4.09 3.56 2.57 2.62 3.86 3.66 2.58 2.74 3.86 3.49 2.82 2.84]]
sample_cards_vec generating 5000
sample_cards_vec took 0.097 Deals: 3789
n_samples 3789 matching bidding info
Samples after bidding filtering:  3789  Threshold:  0.7
Found 3789 samples for bidding
 3D   0.795 Samples: 200
bidding_rollout - n_samples:  200
bidding_rollout - finished  (200, 64)


FileNotFoundError: Could not find module 'libdds.so' (or one of its dependencies). Try using the full path with constructor syntax.

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [ ]:
### Samples consistent with the auction so far

for i in range(min(10,len(bid.samples))):
    print(bid.samples[i])


each row is one sample. the hands are in the order: N E S W

(West always has the same cards because it's your hand and it's known to you)

In [ ]:
# we got some samples above. if we want more, we can always get more

auction = ["PAD_START", "PAD_START", "PAD_START", "PASS", "1N", "PASS", "3S","PASS"]
sample_hands, sorted_score, p_hcp, p_shp, good_quality = bot_bid.sample_hands_for_auction(auction,0)
print("Good quality:", good_quality)
for i in range(min(10,sample_hands.shape[0])):
    print(f'{hand_to_str(sample_hands[i, 0, :])} {hand_to_str(sample_hands[i, 1, :])} {hand_to_str(sample_hands[i, 2, :])} {hand_to_str(sample_hands[i, 3, :])} {sorted_score[i]:.2f}')